# Overview

This notebook demostrates the usage of `piper_control`, our lightweight wrapper
of `piper_sdk` for controlling AgileX Piper robots.

The `piper_sdk` API is powerful and quickly maturing, but it's a bit complex and
under-documented, and we found it helpful to define a simple abstraction for
basic I/O.

There are also several sharp bits in `piper_sdk` which can make the robots seem
tempermental, e.g. becoming unresponsive despite repeated calls to
`MotionCtrl_2`, `EnableArm`, `GripperCtrl`, etc. We've bundled our solutions
into `PiperControl` so `reset` and the various move commands perform as one
would expect.

In [1]:
# Enable autoreload (optional, use if modifying piper_control.py)
%load_ext autoreload
%autoreload 2

# CAN Connect
This section demonstrates the use of `piper_connect` to establish the CAN
connection to the robot(s).

It is a python counterpart to AgileX's `can_activate.sh`.

## Sudo access

This cell is only required for granting sudo to `piper_connect.activate`.

`piper_connect` uses `ip` and `ethtool` commands to manage CAN interfaces, and
those tools often require sudo permissions. For this notebook to get sudo
access, you need to run this cell.

### (Optional) passwordless sudo

For deployed robot machines where you don't want to constantly type in your sudo
password, consider adding your user to sudoers file:

```bash
sudo visudo
# Add this line at the end:
<your username> ALL=(ALL) NOPASSWD:ALL
```

### Grant sudo to notebook

In [ ]:
import subprocess
import getpass

def sudo_warmup():
    try:
        print("🔐 Sudo setup for Jupyter...")
        pw = getpass.getpass("Enter your password for sudo: ")
        proc = subprocess.run(
            ["sudo", "-S", "echo", "sudo access granted"],
            input=pw + "\n", text=True, check=True, capture_output=True
        )
        print(proc.stdout.strip())
    except subprocess.CalledProcessError as e:
        print("❌ Sudo failed:", e.stderr.strip())

sudo_warmup()


## Poll and Connect

In [2]:
from piper_control import piper_connect

In [3]:
ports = piper_connect.find_ports()
print(ports)

[('can0', '1-3:1.0')]


In [4]:
piper_connect.activate()
print(piper_connect.active_ports())

['can0']


# Initialize and Query State

In [5]:

from piper_control import piper_control

robot = piper_control.PiperControl(can_port="can0")

# Resets the robot and enables the motors and motion controller for the arm.
# This call is necessary to be able to both query state and send commands to the
# robot.
robot.reset()

print(robot.get_joint_positions())
print(robot.get_joint_velocities())
print(robot.get_joint_efforts())
print(robot.get_gripper_state())

can0  is exist
can0  is UP
can0  bitrate is  1000000
can0 bus opened successfully.
Enable status: False
Enable status: False
Enable status: True
Arm enabled: True
Motion enabled: True
✅ Finished enabling
[0.005076204, -0.030701440000000003, 0.03914433600000001, -0.123172084, 0.340158, 1.3174929880000001]
[0.509, 0.015, 0.007, 0.478, 0.017, 0.0]
[0.0118125, -0.0023625, -0.05197499999999999, 0.00575064, 0.00670908, -0.00095844]
(0.0, 0.004)


In [ ]:
# Prints out a higher level status of the robot.
print(robot.get_status())

## Resetting and enabling the robot

In [ ]:
# Sometimes, calling `reset()` doesn't work the first time and will throw an
# error. In those instances, you can call reset() again and it should work.
# If it still doesn't work, try turning the robot off and on again.
# And if that _still_ doesn't work, take a look at the CAN connection.

# The robot may physically drop when calling this, so move it to a safe position
# or hold it when calling this.
robot.reset()

## Move a joint

In [ ]:
joint_angles = robot.get_joint_positions()
joint_angles[-2] -= 0.1
print(f"setting joint angles to {joint_angles}")
robot.set_joint_positions(joint_angles)

# Send Gripper Commands

In [ ]:
robot.set_gripper_ctrl(piper_control.GRIPPER_ANGLE_MAX * 0.9, piper_control.GRIPPER_EFFORT_MAX * 0.5)

In [ ]:
robot.set_gripper_ctrl(piper_control.GRIPPER_ANGLE_MAX * 0.1, piper_control.GRIPPER_EFFORT_MAX * 0.5)

## Move to special positions

In [ ]:
# TODO(jscholz): Revisit the rest position.
robot.set_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot.set_joint_positions(piper_control.DOWN_POSITION)

## Cartesian Control

In [ ]:
ee_pose = robot.get_end_effector_pose()
ee_pose[0] += 0.02  # Move X by 5 cm
print(f"Sending updated pose: {ee_pose}")
robot.set_cartesian_position(ee_pose)

## Puppeteering

In [ ]:
robot0 = piper_control.PiperControl(can_port="can0")
robot1 = piper_control.PiperControl(can_port="can1")

In [ ]:
robot0.reset()
robot1.reset()

In [ ]:
robot0.set_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot0.set_joint_positions(piper_control.DOWN_POSITION)

In [ ]:
robot1.set_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot1.set_joint_positions(piper_control.DOWN_POSITION)

In [ ]:
import time
while True:
  joint_angles = robot1.get_joint_positions()
  print(f"{joint_angles=}")
  time.sleep(0.01)
  robot0.set_joint_positions(joint_angles)

# Re-zero Last Joint
This joint is designed to be re-zeroed after changing the EOAT

In [ ]:
# Run this cell a few times until the last joint value is 0.0
robot.disable()
print(robot.get_joint_positions())
robot.piper.JointConfig(joint_num=6, set_zero=0xAE)
print(robot.get_joint_positions())

[0.005076204, -0.030701440000000003, 0.03914433600000001, -0.14600628, 0.340280108, 0.0]
[0.005076204, -0.030701440000000003, 0.03914433600000001, -0.14600628, 0.340280108, 0.0]
